# UN General Assembly Resolutions

This notebook draws from a dataset of UN Member States' historical voting records on General Assembly resolutions from 1946 to 2023, and displays preliminary explorations into relationships and trends exhibited by the Member States.

Voting records were sourced from the UN Digital Library (Dag Hammarskjöld Library). Some earlier records on the database do not disclose breakdowns of Members States' voting behaviour. From a starting point of 7,070 resolutions passed by the UN General Assembly (UNGA) between 1946 and 2023 (inclusive), a total of 1,660 from between 1946 and 1982 do not reveal the individual voting records.

Of the 5,410 records that do provide breakdowns by country, 8 records from 2023 are absent from this dataset, leaving a total of 5,402 included in the dataset examined here.

## Data collection

Data collection for this dataset is undertaken via the the `src/segments.py` and `src/records.py` files within this project, which retrieved links to records and then the records themselves from the UN Digital Library.

These files have been written such that the records and dataset may be updated with new resolution vote records that result from future UNGA sessions.

The retrieval process generates CSV and text files inside the project's `src/data` folder as follows, each of which is timestamped to retain previous versions:
- `"{YYYYMMDD_HHMM}_links.csv"`: generated by `src/segments.py`, this is a list of the URL link segments for each UNGA resolution.
- `"{YYYYMMDD_HHMM}_processed_segments.txt"`: generated by `src/records.py`, this keeps track of which resolution records have previously been processed.
- `"{YYYYMMDD_HHMM}_records.csv"`: generated by `src/records.py`, this is the full UNGA resolution dataset, with basic pre-processing applied to split and sort data for further processing.e

## Data pre-processing

The majority of data pre-processing for this project's dataset is undertaken via the `src/preprocessing.py` file.

Some initial pre-processing takes place in `src/records.py`, in the form of converting the raw data to strings or integers, sorting countries within vote lists, and arranging columns for further processing.

More involved processing then takes place in `src/preprocessing.py`, which expands the vote records for each resolution to populate columns for each individual Member State, and consolidates Member States based on the conditions desribed in `README.md`. It is also at this stage that resolution dates are converted to a unified format, and that resolutions are sorted by date and then reference number.

The `src/preprocessing.py` file also drops certain columns, which are retained in the `"YYYYMMDD_HHMM_records.csv"` files in case of future requirement, but are not necessary for this project. The "Record URL" and "Segment" columns are added during the retrieval process, and record the full URL and URL segment of each resolution. The "Meeting Record", "Draft Resolution", "Committee Report", and "Note" columns come from the raw data, but are not required for this exercise.

The pre-processing stage generates the following file:
- `"{YYYYMMDD_HHMM}_UNGA_votes.csv"`: generated by `src/preprocessing.py`, this is the dataset output that results from the retrieval and preprocessing stages described above.

## Dataset overview

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("./data/20240104_1523_UNGA_votes.csv", low_memory=False)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5402 entries, 0 to 5401
Columns: 211 entries, Resolution to ZIMBABWE
dtypes: int64(5), object(206)
memory usage: 8.7+ MB


In [11]:
df.shape

(5402, 211)

This is a large dataset, consisting of 5,402 rows (UNGA resolution records) and 211 columns.

The initial dataset is too large to conveniently display in this notebook, so we will begin by reviewing the first 12 columns; these contain the main details for the resolutions.

### Dataset overview - UNGA Resolution details

In [16]:
pd.concat([df.head(1), df.tail(1)]).iloc[:, :12]

,Resolution,Vote Date,Num Yes,Num No,Num Abstentions,Num Non-Voting,Total Votes,Title,Yes Votes,No Votes,Abstentions,Non-Voting
0,A/RES/17(I)[RULE-73],1946-01-26,25,18,0,8,51,Amendments to the provisional rules of procedu...,"['ARGENTINA', 'BELGIUM', 'BOLIVIA', 'BRAZIL', ...","['AUSTRALIA', 'BELARUS', 'CANADA', 'CHILE', 'C...",[],"['EL SALVADOR', 'ETHIOPIA', 'GREECE', 'GUATEMA..."
5401,A/RES/78/251,2023-12-22,137,2,22,32,193,Proposed programme budget for 2024 : Section 2...,"['ALGERIA', 'ANGOLA', 'ARGENTINA', 'ARMENIA', ...","['AUSTRALIA', 'ISRAEL']","['ALBANIA', 'ANDORRA', 'CAMEROON', 'CANADA', '...","['AFGHANISTAN', 'ANTIGUA AND BARBUDA', 'BELIZE..."


In [22]:
resolution_details_df = df.iloc[:, :12]
resolution_details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5402 entries, 0 to 5401
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Resolution       5402 non-null   object
 1   Vote Date        5402 non-null   object
 2   Num Yes          5402 non-null   int64 
 3   Num No           5402 non-null   int64 
 4   Num Abstentions  5402 non-null   int64 
 5   Num Non-Voting   5402 non-null   int64 
 6   Total Votes      5402 non-null   int64 
 7   Title            5402 non-null   object
 8   Yes Votes        5402 non-null   object
 9   No Votes         5402 non-null   object
 10  Abstentions      5402 non-null   object
 11  Non-Voting       5402 non-null   object
dtypes: int64(5), object(7)
memory usage: 506.6+ KB


A read-out of the first 12 columns of the first and last entries of the dataset is printed above, followed by information on the 12 columns.

The dataset is date-sorted (as described earlier) and contains resolution records from 26 January 1946 to 22 December 2023.

The columns are as follows:
- Resolution: the reference number of the resolution
- Vote Date: the date of the resolution, in YYYY-MM-DD format
- Num Yes: the number of UN Member States that voted in favour of the resolution
- Num No: the number of UN Member States that voted against the resolution
- Num Abstentions: the number of UN Member States that abstained from voting on the resolution
- Num Non-Voting: the number of UN Member States that were not eligible to vote on the resolution
- Total Votes: the sum total of the Num Yes, Num No, Num Abstentions, and Num Non-Voting figures
- Title: the title of the resolution
- Yes Votes: the list of UN Member States that voted in favour of the resolution
- No Votes: the list of UN Member States that voted in favour of the resolution
- Abstentions: the list of UN Member States that abstained from voting on the resolution
- Non-Voting: the list of UN Member States that were not eligible to vote on the resolution

#### Note on vote figures

To become a UNGA resolution, a draft resolution must receive a majority of Yes votes in order to pass. Under current rules this "majority" is typically 50% of Yes and No votes plus one vote, but in some circumstances a two-thirds majority may be required.

This dataset is compiled from UNGA resolutions, and so naturally the number of Yes votes must exceed the No votes, Abstentions, and Non-Voting entries. As subset below shows, there are no resolution entries for which the number of Yes votes is less than or equal to 50% of the number of Yes and No votes combined.

In [36]:
df[df["Num Yes"] / (df["Num Yes"] + df["Num No"]) <= 0.5]

,Resolution,Vote Date,Num Yes,Num No,Num Abstentions,Num Non-Voting,Total Votes,Title,Yes Votes,No Votes,...,URUGUAY,UZBEKISTAN,VANUATU,VENEZUELA,VIETNAM,YEMEN,YEMEN (PDR),YUGOSLAVIA,ZAMBIA,ZIMBABWE


### Dataset overview - UN Member State columns

The original UN records contain the UNGA voting records of 255 current and former Member States.

Those 255 entities are consolidated down to 199 via the process described in this project's `README.md` file, typically by merging Member States that are the same entity but have changed their official names, but also by assessing political changes that have resulted in the merger or dissolution of Member States.

See `README.md` for further details of the vote record consolidation.

Of the 199 consolidated entities referred to above, 193 are current Member States as of January 2024. The former Member States for whcih is has not been possible to align their records with current Member States are Czechoslovakia, East Germany, Federation of Malaya, Serbia and Montenegro, Yemen (PDR), and Yugoslavia.

The records for those entities feature significant gaps prior to and/or following their tenure as Member States; this is true also of those Member States that acceeded to the UN subsequent to its establishment. South Sudan, the most recent entry to the UNGA, joined in 2011, and has votes recorded for just 1,097 (20.3%) of the 5,402 resolutions mentioned above.